In [1]:
import numpy as np
import random
import copy

In [2]:
#問題生成
def puzzle_generation(split_list, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()

    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]
    row = l0[1]
    selection_cost = l0[2]

    for i in range(1, row+1):
        row_data = data[i].rstrip('\n').rstrip(',')
        line_data = [int(x.strip()) for x in row_data.split(',')]
        row_array = np.array(line_data)
        if i == 1:
            split_list = row_array
        else:
            split_list = np.vstack((split_list, row_array))

    print('colmun(列→)：' + str(colmun))
    print('row(行↓)：' + str(row))
    print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')


    #ランダム生成
    # random_num = random.sample(range(1, colmun*row+1), colmun*row)
    # random_list = np.array(random_num)
    # split_list = random_list.reshape(colmun, row)

    return split_list

In [3]:
split_list =  np.array([])
split_list = puzzle_generation(split_list, 'issue/4x4.txt')
print(split_list)

colmun(列→)：4
row(行↓)：4
selection_cost(選択コスト)：2

[[11  9  5  8]
 [12  4  7 10]
 [15  3  1 13]
 [14  2 16  6]]
